In [ ]:
from pulp import *
import pandas as pd
#Heavily influenced by, need to make sure we understand the code so we can use it and not plagerize as a final product
#https://www.kaggle.com/code/itoeiji/solving-tsp-and-vrp-by-mip-using-pulp

In [ ]:
distEnum = {
    "SFO": { "SFO": 0, "LAX": 500, "SJC": 75, "OAK": 50},
    "LAX": { "SFO": 500, "LAX": 0, "SJC": 280, "OAK": 800},
    "SJC": { "SFO": 75, "LAX": 280, "SJC": 0, "OAK": 30},
    "OAK": { "SFO": 50, "LAX": 800, "SJC": 30, "OAK": 0}
}


distDict = pd.DataFrame( 
    [
    [0, 500, 75, 50],
    [500, 0, 280, 800],
    [75, 280, 0, 30],
    [50, 800, 30, 0]
    ])

distDict2 = pd.DataFrame( 
    [
    [0, 9999, 469, 857, 375],
    [9999, 0, 549, 1188, 464],
    [469, 549, 0, 639, 380],
    [857, 1188, 639, 0, 9999],
    [375, 464, 380, 9999, 0]
    ])

In [50]:
usRoutes = pd.read_csv('../data/usRoutesClean.csv')

In [54]:
usRoutes['SRCIATA'].sample(n=5)

3772    MSY
3559    IND
1710    ATL
4652    FLL
540     CMH
Name: SRCIATA, dtype: object

In [19]:
len(distDict)

4

In [43]:
problem = LpProblem("LP_Test", LpMinimize)

In [44]:
x = LpVariable.dicts('x', ((i, j) for i in range(len(distEnum)) for j in range(len(distEnum))), lowBound=0, upBound=1, cat='Binary')

In [45]:
problem += pulp.lpSum(distDict[i][j] * x[i, j] for i in range(len(distEnum)) for j in range(len(distEnum)))

In [46]:
u = pulp.LpVariable.dicts('u', (i for i in range(len(distEnum))), lowBound=1, upBound=800, cat='Integer')

In [47]:
for i in range(len(distEnum)):
    problem += x[i, i] == 0

for i in range(len(distEnum)):
    problem += pulp.lpSum(x[i, j] for j in range(len(distEnum))) == 1
    problem += pulp.lpSum(x[j, i] for j in range(len(distEnum))) == 1

In [48]:
for i in range(len(distEnum)):
    for j in range(len(distEnum)):
        if i != j and (i != 0 and j != 0):
            problem += u[i] - u[j] <= len(distEnum) * (1 - x[i, j]) - 1

In [49]:
status = problem.solve()
status, LpStatus[status], value(problem.objective)

(1, 'Optimal', 860.0)